
## This Notebook is used to filter out or drop the duplicates from the files 

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime 

In [0]:
#run the notebook setup notebookk

In [0]:
%run /Workspace/Generic/Nb_SetUpMounts_JDBC

[FileInfo(path='dbfs:/FileStore/2024_04_18_4_WheelSales-1.csv', name='2024_04_18_4_WheelSales-1.csv', size=10992761, modificationTime=1719993217000),
 FileInfo(path='dbfs:/FileStore/2024_04_18_4_WheelSales.csv', name='2024_04_18_4_WheelSales.csv', size=10992762, modificationTime=1719989742000),
 FileInfo(path='dbfs:/FileStore/4_WheelSales.csv', name='4_WheelSales.csv', size=10992762, modificationTime=1719989774000),
 FileInfo(path='dbfs:/FileStore/jars/', name='jars/', size=0, modificationTime=1716203553000),
 FileInfo(path='dbfs:/FileStore/source1.csv', name='source1.csv', size=1138, modificationTime=1709555634000),
 FileInfo(path='dbfs:/FileStore/tables/', name='tables/', size=0, modificationTime=1708600418000)]

['PipelineName', 'Pk_PipelineID', 'Pk_ActivityID', 'PipelineSourceTypes', 'ParameterValueSink', 'parameterValues2', 'ColumnSourceTypes', 'SourceColumnName', 'TargetColumnName', 'sourcecolumntypes', 'IsNullable', 'IsActive']


PipelineName,Pk_PipelineID,Pk_ActivityID,PipelineSourceTypes,ParameterValueSink,parameterValues2,ColumnSourceTypes,SourceColumnName,TargetColumnName,sourcecolumntypes,IsNullable,IsActive
PL_Copy2WheelsData,101,2,CSV,2-WheelSales.csv,2-WheelSData.parquet,CSV,"[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, Model Price, Dealer location, Region]","[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, ModelPrice, Dealerlocation, Region]","[string, string, string, string, int, int, string, string, string, string, string]",0,1
PL_Copy4WheelsData,102,3,CSV,4-WheelSales.csv,4-WheelSData.parquet,CSV,"[Brand, Model, Body Type, FUEL TYPE, Mileage, ENGINE, TRANSMISSION, Price range, Fuel Capacity (L), Seating Capacity, Top_Speed in Km/h, No Of Cylinders, Year Of Sale, Month Of Sale , Country, City]","[Brand, Model, BodyType, FUELTYPE, Mileage, ENGINE, TRANSMISSION, Pricerange, FuelCapacity, SeatingCapacity, Top_Speed, NoOfCylinders, YearOfSale, MonthOfSale, Country, City]","[string, string, string, string, string, string, string, string, int, int, string, int, int, string, string, string]",null,null


PipelineName,Pk_PipelineID,Pk_ActivityID,PipelineSourceTypes,ParameterValueSink,parameterValues2,ColumnSourceTypes,SourceColumnName,TargetColumnName,sourcecolumntypes,IsNullable,IsActive,test
PL_Copy2WheelsData,101,2,CSV,2-WheelSales.csv,2-WheelSData.parquet,CSV,"[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, Model Price, Dealer location, Region]","[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, ModelPrice, Dealerlocation, Region]","[string, string, string, string, int, int, string, string, string, string, string]",0,1,2-WheelSales
PL_Copy4WheelsData,102,3,CSV,4-WheelSales.csv,4-WheelSData.parquet,CSV,"[Brand, Model, Body Type, FUEL TYPE, Mileage, ENGINE, TRANSMISSION, Price range, Fuel Capacity (L), Seating Capacity, Top_Speed in Km/h, No Of Cylinders, Year Of Sale, Month Of Sale , Country, City]","[Brand, Model, BodyType, FUELTYPE, Mileage, ENGINE, TRANSMISSION, Pricerange, FuelCapacity, SeatingCapacity, Top_Speed, NoOfCylinders, YearOfSale, MonthOfSale, Country, City]","[string, string, string, string, string, string, string, string, int, int, string, int, int, string, string, string]",null,null,4-WheelSales


/mnt/genericGold/2-WheelSales
2-WheelSales
abfss://xyenta-dev-96-gold@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
/mnt/genericGold/4-WheelSales
4-WheelSales
abfss://xyenta-dev-96-gold@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


2-WheelSales
/mnt/genericSilver/2-WheelSales
abfss://xyenta-dev-96-silver@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
4-WheelSales
/mnt/genericSilver/4-WheelSales
abfss://xyenta-dev-96-silver@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


2-WheelSales
/mnt/genericrawTest/2-WheelSales
abfss://xyenta-dev-96-raw@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
4-WheelSales
/mnt/genericrawTest/4-WheelSales
abfss://xyenta-dev-96-raw@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


2-WheelSales
/mnt/genericStage/2-WheelSales
abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
4-WheelSales
/mnt/genericStage/4-WheelSales
abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


In [0]:
## define a function which drops the duplicates 

In [0]:
def fn_drop_duplicates(filepaths, reference_df):
    try:
        for x in reference_df.collect():
            filenames = x['parameterValues2']
            folder_names = x['test']

            dt_string = datetime.now().strftime("year=%Y/Month=%m/day=%d")
            full_path = f'{filepaths}{folder_names}/{dt_string}/'
            print(f'Checking path: {full_path}')

            try:
                dbutils.fs.ls(full_path)
                print(f'Exists: {full_path}')

                dfs = spark.read.format('parquet').option('inferSchema', True).load(full_path)
                print("Schema before dropping duplicates:")
                dfs.printSchema()

                original_count = dfs.count()
                print(f'Original record count for {filenames}: {original_count}')

                dfs_cleaned = dfs.dropDuplicates()
                cleaned_count = dfs_cleaned.count()
                print(f'Record count after dropping duplicates for {filenames}: {cleaned_count}')

                dt_string2 = datetime.now().strftime("year=%Y/Month=%m/day=%d")
                output_good_path = f"/mnt/genericSilver/{folder_names}/{dt_string2}"
                
                print(f'Writing cleaned data to: {output_good_path}')
                dfs_cleaned.coalesce(1).write.mode('overwrite').option("header", "true").parquet(output_good_path)
                print("Write operation successful")

            except Exception as e:
                print(f'Error processing path {full_path}: {e}')
                continue

    except Exception as e:
        raise e


In [0]:
fn_drop_duplicates(filepaths= '/mnt/genericSilver/', reference_df=res)

Checking path: /mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=09/
Exists: /mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=09/
Schema before dropping duplicates:
root
 |-- Dealerlocation: string (nullable = true)
 |-- EngineCapacity: string (nullable = true)
 |-- Mileage: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- ModelPrice: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- NoOfSales: integer (nullable = true)
 |-- OEM: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Year: integer (nullable = true)

Original record count for 2-WheelSData.parquet: 25346
Record count after dropping duplicates for 2-WheelSData.parquet: 23427
Writing cleaned data to: /mnt/genericSilver/2-WheelSales/year=2024/Month=07/day=09
Write operation successful
Checking path: /mnt/genericSilver/4-WheelSales/year=2024/Month=07/day=09/
Exists: /mnt/genericSilver/4-WheelSales/year=2024/Month=07/day=09/
Sche